In [6]:
import ScraperFC as sfc
import traceback
import pandas as pd
pd.options.display.max_columns = 50
from IPython.display import clear_output

In [8]:
scraper = sfc.FBRef()
try:
    out = scraper.scrape_season(2020, "MLS")
    if type(out) is int:
        print(out)
    elif type(out) is tuple:
        print(out[0], out[1])
    elif type(out) is dict:
        print(out.keys())
    else:
        print(out.shape)
    scraper.close()
except:
    print(traceback.format_exc())
    scraper.close()

dict_keys(['League Table', 'Standard', 'Goalkeeping', 'Advanced Goalkeeping', 'Shooting', 'Passing', 'Pass Types', 'GCA', 'Defensive', 'Possession', 'Playing Time', 'Misc'])


In [10]:
out["Possession"][0]

Unnamed: 0_level_0 Unnamed: 1_level_0 Unnamed: 2_level_0  \
                Squad               # Pl               Poss   
0         Atlanta Utd                 27               54.8   
1        Chicago Fire                 25               51.0   
2     Colorado Rapids                 22               52.0   
3       Columbus Crew                 26               52.2   
4         D.C. United                 26               46.0   
5       FC Cincinnati                 30               49.3   
6           FC Dallas                 27               51.4   
7      Houston Dynamo                 26               48.5   
8         Inter Miami                 28               47.1   
9           LA Galaxy                 25               45.0   
10     Los Angeles FC                 28               58.6   
11      Minnesota Utd                 28               47.3   
12    Montreal Impact                 26               48.0   
13          Nashville                 26               44.5   
14        New England                 25               51.2   
15       NY Red Bulls                 27               45.7   
16              NYCFC                 22               56.0   
17       Orlando City                 28               50.3   
18       Philadelphia                 23               47.9   
19   Portland Timbers                 27               46.5   
20     Real Salt Lake                 26               52.3   
21           San Jose                 27               55.7   
22            Seattle                 27               54.5   
23        Sporting KC                 23               49.7   
24         Toronto FC                 29               54.5   
25          Vancouver                 27               41.8   

   Unnamed: 3_level_0 Touches                                                 \
                  90s Touches Def Pen Def 3rd Mid 3rd Att 3rd Att Pen   Live   
0                23.0   14792    1565    5080    7267    3442     429  13746   
1                23.0   13921    1513    4754    6490    3477     582  12785   
2                18.0   11267    1184    3717    5415    2927     395  10512   
3                23.0   13963    1420    4379    6650    3785     507  12896   
4                23.0   13373    1566    5099    6229    2798     357  12268   
5                23.0   14009    1581    5892    6405    2566     328  12918   
6                22.0   13548    1704    5252    6100    3050     517  12514   
7                23.0   13898    1629    5058    5993    3747     567  12799   
8                23.0   12678    1118    3866    5930    3664     565  11566   
9                22.0   12064    1242    3870    5443    3452     502  11056   
10               22.0   15220    1141    3894    7351    4901     707  14095   
11               21.0   12479    1292    3902    5673    3634     470  11493   
12               23.0   13522    1586    4614    6379    3400     454  12439   
13               23.0   12976    1403    4394    6299    3060     424  11990   
14               23.0   13989    1280    4159    6688    3899     588  12751   
15               23.0   13030    1122    3536    5939    4182     502  11812   
16               23.0   15044    1505    4879    6832    4305     690  13904   
17               23.0   13721    1474    4696    6542    3284     485  12683   
18               23.0   13119    1394    4151    5960    3790     546  12022   
19               23.0   13504    1694    4961    6107    3279     531  12433   
20               22.0   13255    1307    4509    6357    3217     494  12200   
21               23.0   14349    1687    5232    6467    3617     509  13250   
22               22.0   14681    1440    4666    6933    3971     585  13724   
23               21.0   12556    1123    3724    5649    4093     614  11527   
24               23.0   15757    1471    4838    7654    4264     604  14675   
25               23.0   12412    1818    5125    5546    2483     351  11390  

In [ ]:
# scraper = sfc.Understat()
# broke = False
# shapes = list()
# for yr in range(2017,2022):
#     for lg in ['EPL','La Liga','Bundesliga','Serie A','Ligue 1']:

#         try:
#             out = scraper.scrape_shot_xy(yr, lg)
#             shapes.append([yr, lg, len(out)])
#         except:
#             traceback.print_exc()
#             scraper.close()
#             broke = True
#             break

#     if broke:
#         break
# scraper.close()

In [ ]:
# scraper = sfc.WhoScored()

# try:
#     out = scraper.scrape_matches(2020, "EPL")
#     print(out)
#     # scraper.close()
# except:  
#     traceback.print_exc()
#     # scraper.close()

In [ ]:
# import json
# test = \
#     '{\n \
#         "matchId":1376072,\n \
#         "matchCentreData": \
#         { \
#             "playerIdNameDictionary": \
#             { \
#                 "81523":"Mat Ryan", \
#                 "113994":"Leandro Trossard", \
#                 "362146":"Steven Alzate", \
#                 "119822":"Neal Maupay", \
#                 "71824":"Pascal Groß", \
#                 "344325":"Aaron Connolly", \
#                 "79442":"Ezequiel Schelotto", \
#                 "303096":"Yves Bissouma", \
#                 "77429":"Davy Pröpper", \
#                 "43362":"Leon Balogun", \
#                 "91387":"Martín Montoya", \
#                 "109922":"Adam Webster", \
#                 "122926":"Solly March", \
#                 "82277":"Dan Burn", \
#                 "86441":"Lewis Dunk", \
#                 "21427":"Glenn Murray", \
#                 "24238":"Dale Stephens", \
#                 "34388":"David Button", \
#                 "101642":"Bernard", \
#                 "317804":"Richarlison", \
#                 "131171":"Jonas Lössl", \
#                 "89925":"Lucas Digne", \
#                 "29544":"Morgan Schneiderlin", \
#                 "10454":"Djibril Sidibé", \
#                 "69346":"Gylfi Sigurdsson", \
#                 "31826":"Séamus Coleman", \
#                 "328990":"Moise Kean", \
#                 "136824":"Alex Iwobi", \
#                 "118123":"André Gomes", \
#                 "13796":"Theo Walcott", \
#                 "27213":"Fabian Delph", \
#                 "141469":"Dominic Calvert-Lewin", \
#                 "297544":"Mason Holgate", \
#                 "316077":"Tom Davies", \
#                 "107941":"Michael Keane", \
#                 "110189":"Jordan Pickford" \
#             }  \
#         } \
#     }'
# temp = out.split("require.config.params[\"args\"] =")[1] \
#     .split("</script>")[0] \
#     .strip() \
#     .split("\n")
# new_temp = str()
# for segment in temp[1:-1]:
#     # print(segment[-50:])
#     segment = segment.strip()
#     segment = "\"" + segment
#     segment = segment.split(":")
#     segment = "\":".join(segment[:2]) + ":".join(segment[2:])
#     # print(segment[-50:])
#     new_temp += segment
# new_temp = new_temp+"}"
# json.loads(new_temp)